In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time

In [2]:
def TDT(n):
    b = n[0] + n[2] + 2*n[3]
    c = n[1] + n[2] + 2*n[4]
    
    if b == 0 and c == 0:
        return 0
    elif b == 0:
        return 1000000000
    else:
        return (b-c)**2/(b+c)

def SHD(cc, n):
    T = TDT(n)
    
    d = np.zeros(2)
    N = np.zeros(6)
    
    for k in range(6):
        N[k] = n[k]
    
    if T < cc:
        while T < cc:
            if N[4] > 0:
                N[4] -= 1
            elif N[1] > 0:
                N[1] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[0] -= 1
            N[3] += 1
            
            T = TDT(N)
            d[0] -= 1
        
        T = TDT(n)
        for k in range(6):
            N[k] = n[k]
        
        while T < cc:
            if N[3] > 0:
                N[3] -= 1
            elif N[0] > 0:
                N[0] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[1] -= 1
            N[4] += 1
            
            T = TDT(N)
            d[1] -= 1
        
        if d[0] > d[1]:
            return d[0]
        else:
            return d[1]
    
    else:
        step = 0
        
        if n[0] + 2*n[3] > n[1] + 2*n[4]:
            while T >= cc:
                if n[3] > 0:
                    n[3] -= 1
                elif n[0] > 0:
                    n[0] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[1] -= 1
                n[4] += 1
            
                T = TDT(n)
                step += 1
        else:
            while T >= cc:
                if n[4] > 0:
                    n[4] -= 1
                elif n[1] > 0:
                    n[1] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[0] -= 1
                n[3] += 1
            
                T = TDT(n)
                step += 1
        
        return step-1

In [3]:
def appx_SHD(cc,n):
    b = n[0] + n[2] + 2*n[3]
    c = n[1] + n[2] + 2*n[4]
    
    T = TDT(n)
    
    s = b + c
    d = math.fabs(b-c)
    
    if T >= cc:
        return math.ceil((d-math.sqrt(s*cc)) / 4) - 1
    
    else:
        if b + c < cc:
            return -math.ceil((2*cc - s - d)/4)
        else:
            return -math.ceil((math.sqrt(s*cc)-d)/4)

In [4]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_mec(shd, m, K, epsilon):
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * shd[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        shd[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [5]:
# b, cを与える (n0=b, n1=cとする, n5 = 2N-b-c)
# N = 150, M = 5000

for j in range(5):
    N = 150
    M = 5000
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    
    n = np.zeros((M,6))
    
    for i in range(M-10):
        S[i] = random.randint(0,2*N)
        b[i] = np.random.binomial(S[i],0.5)
        c[i] = S[i] - b[i]
        n[i][0] = b[i]
        n[i][1] = c[i]
        n[i][5] = 2*N - b[i] - c[i]
    
    for i in range(M-10, M):
        S[i] = random.randint(0,2*N)
        b[i] = np.random.binomial(S[i],0.75)
        c[i] = S[i] - b[i]
        n[i][0] = b[i]
        n[i][1] = c[i]
        n[i][5] = 2*N - b[i] - c[i]
    
    shd = np.zeros(M)
    appx_shd = np.zeros(M)
    cc = 19.5

    appx_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        appx_shd[i] = appx_SHD(cc,k)
    appx_end = time.time()
    
    #print(shd)
    #print(appx_shd)
    
    print("appx_time =",appx_end-appx_start,"[sec]")
    
    exact_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        shd[i] = SHD(cc,k)
    exact_end = time.time()
        
    print("exact_time =",exact_end-exact_start,"[sec]")
    print("----------")

appx_time = 0.02395319938659668 [sec]
exact_time = 0.8906440734863281 [sec]
----------
appx_time = 0.01874685287475586 [sec]
exact_time = 0.8671979904174805 [sec]
----------
appx_time = 0.018947124481201172 [sec]
exact_time = 0.8724191188812256 [sec]
----------
appx_time = 0.019025087356567383 [sec]
exact_time = 0.8733620643615723 [sec]
----------
appx_time = 0.019557952880859375 [sec]
exact_time = 0.8693041801452637 [sec]
----------


In [6]:
# b, cを与える (n0=b, n1=cとする, n5 = 2N-b-c)
#N = 5000, M = 1000000

for j in range(5):
    N = 5000
    M = 1000000
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    
    n = np.zeros((M,6))
    
    for i in range(M-10):
        S[i] = random.randint(0,2*N)
        b[i] = np.random.binomial(S[i],0.5)
        c[i] = S[i] - b[i]
        n[i][0] = b[i]
        n[i][1] = c[i]
        n[i][5] = 2*N - b[i] - c[i]
    
    for i in range(M-10, M):
        S[i] = random.randint(0,2*N)
        b[i] = np.random.binomial(S[i],0.55)
        c[i] = S[i] - b[i]
        n[i][0] = b[i]
        n[i][1] = c[i]
        n[i][5] = 2*N - b[i] - c[i]
    
    shd = np.zeros(M)
    appx_shd = np.zeros(M)
    cc = 29.7

    appx_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        appx_shd[i] = appx_SHD(cc,k)
    appx_end = time.time()
    
    #print(shd)
    #print(appx_shd)
    
    print("appx_time =",appx_end-appx_start,"[sec]")
    
    exact_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        shd[i] = SHD(cc,k)
    exact_end = time.time()
        
    print("exact_time =",exact_end-exact_start,"[sec]")
    print("----------")

appx_time = 4.072306871414185 [sec]
exact_time = 1088.0571348667145 [sec]
----------
appx_time = 4.044203042984009 [sec]
exact_time = 1083.4222741127014 [sec]
----------
appx_time = 4.061657905578613 [sec]
exact_time = 1068.2421612739563 [sec]
----------
appx_time = 3.9687998294830322 [sec]
exact_time = 1085.5185761451721 [sec]
----------
appx_time = 4.086510896682739 [sec]
exact_time = 1083.625060081482 [sec]
----------


In [7]:
def generate_n_small(N, M):
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    n = np.zeros((M,6))
    
    for i in range(M-10):
        S[i] = 2*N
        n[i][0] = np.random.binomial(S[i],1/6)
        n[i][1] = np.random.binomial((S[i]-n[i][0]), 1/5)
        n[i][2] = np.random.binomial((S[i]-n[i][0]-n[i][1]), 1/4)
        n[i][3] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]), 1/3)
        n[i][4] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]), 1/2)
        n[i][5] = S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]-n[i][4]
    
    for i in range(M-10,M):
        S[i] = 2*N
        n[i][0] = np.random.binomial(S[i],1/4)
        n[i][1] = np.random.binomial((S[i]-n[i][0]), 1/8)
        n[i][2] = np.random.binomial((S[i]-n[i][0]-n[i][1]), 1/4)
        n[i][3] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]),1/2)
        n[i][4] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]), 1/3)
        n[i][5] = S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]-n[i][4]
    
    return n

def generate_n_large(N, M):
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    n = np.zeros((M,6))
    
    for i in range(M-10):
        S[i] = 2*N
        n[i][0] = np.random.binomial(S[i],1/6)
        n[i][1] = np.random.binomial((S[i]-n[i][0]), 1/5)
        n[i][2] = np.random.binomial((S[i]-n[i][0]-n[i][1]), 1/4)
        n[i][3] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]), 1/3)
        n[i][4] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]), 1/2)
        n[i][5] = S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]-n[i][4]
    
    for i in range(M-10,M):
        S[i] = 2*N
        n[i][0] = np.random.binomial(S[i],11/60)
        n[i][1] = np.random.binomial((S[i]-n[i][0]), 2/11)
        n[i][2] = np.random.binomial((S[i]-n[i][0]-n[i][1]), 1/4)
        n[i][3] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]), 11/30)
        n[i][4] = np.random.binomial((S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]), 5/11)
        n[i][5] = S[i]-n[i][0]-n[i][1]-n[i][2]-n[i][3]-n[i][4]
    
    return n

In [8]:
# n0~n5を与える
# N = 150,M = 5000, 時間計測

for j in range(5):
    N = 150
    M = 5000
    b = np.zeros(M)
    c = np.zeros(M)
    
    n = np.zeros((M,6))
    
    n = generate_n_small(N,M)
    
    for i in range(M):
        b[i] = n[i][0] + n[i][2] + 2*n[i][3]
        c[i] = n[i][1] + n[i][2] + 2*n[i][4]
    
    shd = np.zeros(M)
    appx_shd = np.zeros(M)
    cc = 19.5
    
    appx_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        appx_shd[i] = appx_SHD(cc,k)
    appx_end = time.time()
    
    print("appx_time =",appx_end-appx_start,"[sec]")
    
    exact_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        shd[i] = SHD(cc,k)
    exact_end = time.time()
    
    print("exact_time =",exact_end-exact_start,"[sec]")
    print("----------")

appx_time = 0.01872706413269043 [sec]
exact_time = 0.9678599834442139 [sec]
----------
appx_time = 0.01877307891845703 [sec]
exact_time = 0.9683499336242676 [sec]
----------
appx_time = 0.018872737884521484 [sec]
exact_time = 0.9795207977294922 [sec]
----------
appx_time = 0.018855810165405273 [sec]
exact_time = 0.9701581001281738 [sec]
----------
appx_time = 0.018784046173095703 [sec]
exact_time = 0.9745559692382812 [sec]
----------


In [9]:
# n0~n5を与える
# N = 5000,M = 1000000, 時間計測

for j in range(5):
    N = 5000
    M = 1000000
    b = np.zeros(M)
    c = np.zeros(M)
    
    n = np.zeros((M,6))
    
    n = generate_n_large(N,M)
    
    for i in range(M):
        b[i] = n[i][0] + n[i][2] + 2*n[i][3]
        c[i] = n[i][1] + n[i][2] + 2*n[i][4]
    
    shd = np.zeros(M)
    appx_shd = np.zeros(M)
    cc = 29.7
    
    appx_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        appx_shd[i] = appx_SHD(cc,k)
    appx_end = time.time()
    
    print("appx_time =",appx_end-appx_start,"[sec]")
    
    exact_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        shd[i] = SHD(cc,k)
    exact_end = time.time()
    
    print("exact_time =",exact_end-exact_start,"[sec]")
    print("----------")

appx_time = 4.079542875289917 [sec]
exact_time = 1303.8181953430176 [sec]
----------
appx_time = 4.026061058044434 [sec]
exact_time = 1330.782441854477 [sec]
----------
appx_time = 4.225023984909058 [sec]
exact_time = 1344.9454028606415 [sec]
----------
appx_time = 4.2678492069244385 [sec]
exact_time = 1358.8221859931946 [sec]
----------
appx_time = 4.216782093048096 [sec]
exact_time = 1353.267260313034 [sec]
----------
